In [1]:
import torch
import pandas as pd
import numpy as np
# pip install transformers
from transformers import BertTokenizer, BertModel
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets

In [4]:
main_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv') # user-item-time
year_df = pd.read_csv('/opt/ml/input/data/train/years.tsv', sep='\t') # item-year
title_df = pd.read_csv('/opt/ml/input/data/train/titles.tsv', sep='\t') # item-title

In [5]:
main_df = pd.merge(main_df, year_df, on='item', how='left')
main_df = pd.merge(main_df, title_df, on='item', how='left')

# 제목의 연도로 연도 결측치 제거
import re
main_df['year_from_title'] = main_df['title'].apply(lambda x : (x[-5:-1]))
# 007-을 2007로 변경
main_df.loc[main_df['year_from_title']=='007-','year_from_title'] = 2007
# year가 결측치인 곳에 삽입
main_df.loc[main_df['year'].isnull(),'year'] = main_df.loc[main_df['year'].isnull(),'year_from_title'].astype('float')

main_df = main_df.drop(['year_from_title'],axis=1)


In [14]:
set(main_df['item']) - set(year_df['item'])

{3310, 6987, 6988, 7065, 7243, 8511, 32898, 119145}

In [15]:
missing_year_item = set(main_df['item']) - set(year_df['item'])

In [21]:
list(main_df[main_df['item'] == 3310]['year'])[0]

1921.0

In [22]:
tmp = []
for item in missing_year_item:
    tmp.append((item, list(main_df[main_df['item'] == item]['year'])[0]))

In [24]:
no_year_item_df = pd.DataFrame(tmp)

In [26]:
no_year_item_df.columns = ['item','year']
no_year_item_df

,item,year
0,32898,1902.0
1,119145,2015.0
2,6987,1920.0
3,6988,1919.0
4,7243,1916.0
5,3310,1921.0
6,7065,1915.0
7,8511,1917.0


In [28]:
result_df = pd.concat([year_df, no_year_item_df])

,item,year
0,1348,1922.0
1,44587,1922.0
2,4768,1922.0
3,8235,1923.0
4,8609,1923.0
...,...,...
3,6988,1919.0
4,7243,1916.0
5,3310,1921.0
6,7065,1915.0


In [29]:
result_df

,item,year
0,1348,1922.0
1,44587,1922.0
2,4768,1922.0
3,8235,1923.0
4,8609,1923.0
...,...,...
3,6988,1919.0
4,7243,1916.0
5,3310,1921.0
6,7065,1915.0


In [31]:
result_df.to_csv(
    "year_fe.csv", index=False
)